In [1]:
using Gridap, Gridap.Geometry, Gridap.Fields
using Gmsh, GridapGmsh
using GLMakie, DelimitedFiles, Interpolations
using LinearAlgebra, SparseArrays, KrylovKit
using ChainRulesCore, Zygote
using PartitionedArrays
using NLopt
using GridapMakie

import Gridap.CellData: Interpolable
import ChainRulesCore: rrule
import Gmsh: gmsh


include("Materials/Materials.jl")
include("Module/Mesh_Periodic.jl")
include("Module/Helper.jl")
include("Module/GridapFE.jl")
include("Module/Control.jl")


LWConstraint (generic function with 1 method)

In [2]:
# Geometry parameters of the mesh
L = 150           # Length of the normal region  
hair = 500        # Height of the air region
hs = 300          # Height of the source location in air
ht = 200          # Height of the target location in air
hd = 200          # Height of design domain
hsub = 100        # Height of substrate domain below design domain
dpml = 100        # Thickness of the PML

# Characteristic length (controls the resolution, smaller the finer)
resol = 20        # Number of points per wavelength
l1 = L/resol      # Air
l2 = l1/2.0       # Design domain
l3 = l1           # PML

meshfile = "geometry.msh"
geo_param = PeriodicGeometry(L, hair, hs, ht, hd, hsub, dpml, l1, l2, l3)
# MeshGenerator(geo_param, meshfile)


PeriodicGeometry(150.0, 500.0, 300.0, 200.0, 200.0, 100.0, 100.0, 7.5, 3.75, 7.5)

In [3]:
############  Optimization parameters #############
flag_f = true       # Turn on filter
flag_t = true       # Turn on threshold

# Filter and threshold paramters
r = [0.02 * L, 0.02 * L]  # Filter radius
β = 80.0                  # β∈[1,∞], threshold sharpness
η = 0.5                   # η∈[0,1], threshold center

α = 0.0 / (2 * 1000.0)    # Equivalent loss α = 1/2Q

# Number of subspace
K = 20

# Amplify g for NLopt
Amp = 1

# Sum over kx
nkx = 30
nparts = nkx / 2

Bp = true          # Matrix B depend on parameters?
pv = 0.5

# Foundary constraint parameters
c = 0#resol^4
lw = r[1]
ls = r[1]
ηe = fηe(lw / r[1])
ηd = fηd(lw / r[1])

control = ControllingParameters(flag_f, flag_t, r, β, η, α, nparts, nkx, K, Amp, Bp, pv, c, ηe, ηd)

gridap = GridapFE(meshfile, 1, 2, ["DirichletEdges", "DirichletNodes"], ["DesignNodes", "DesignEdges"], ["Target"], ["Source"], flag_f)

#phys = PhysicalParameters(k, kb, ω, ϵ1, ϵ2, ϵ3, ϵd, μ, R, σs, dpml, LHp, LHn, wg_center, wg_size)


Info    : Reading 'geometry.msh'...
Info    : 45 entities
Info    : 6325 nodes
Info    : 12418 elements
Info    : Done reading 'geometry.msh'


GridapParameters(UnconstrainedFESpace(), UnconstrainedFESpace(), UnconstrainedFESpace(), UnconstrainedFESpace(), UnconstrainedFESpace(), UnconstrainedFESpace(), 5046, UnstructuredGrid(), Measure(), Measure(), Measure(), Measure(), Measure(), GenericCellField(), Int8[8, 8, 8, 8, 8, 8, 8, 8, 8, 8  …  10, 10, 10, 10, 10, 10, 10, 10, 10, 10], 7)

In [4]:
include("Module/Model.jl")

VectorO (generic function with 1 method)

In [5]:

material = "Gold"
n_λ, k_λ = RefractiveIndex(material)
λ1 = 532
λ2 = 549
nm1 = 0.4+2.5im#n_λ(λ1) + 1im * k_λ(λ1)
nm2 = 0.5#+2.6im#n_λ(λ2) + 1im * k_λ(λ2)
nf = sqrt(1.77)
μ = 1
R = 1e-10
LHp=[Inf, hair + hd]  # Start of PML for x,y > 0
LHn=[Inf, hsub]       # Start of PML for x,y < 0


ω1 = 2 * π / λ1
phys1 = PhysicalParameters(ω1, nf, nm1, nm1, μ, R, dpml, LHp, LHn, hd)
ω2 = 2 * π / λ2
phys2 = PhysicalParameters(ω2, nf, nm2, nm2, μ, R, dpml, LHp, LHn, hd)


# kb = 0. * ω1
# p0 = zeros(gridap.np)
# p_vec = p_extend(p0; gridap)
# pf_vec = Filter(p_vec; control, gridap)
# pf_vec[pf_vec .< 0] .= 0
# pf_vec[pf_vec .> 1] .= 1.0
# pfh = FEFunction(gridap.FE_Pf, pf_vec)
# pth = (pf -> Threshold(pf; control)) ∘ pfh

# A1_mat = MatrixA(pth, kb; phys=phys1, control, gridap)

# btest(v) = ∫(v)gridap.dΓ_s
# b_vec = assemble_vector(btest, gridap.FE_V)

# u1_vec = A1_mat\b_vec
# u1h = FEFunction(gridap.FE_U, u1_vec)
# A2_mat = MatrixA(pth, kb; phys=phys2, control, gridap)
# o_vec = VectorO(1, 1; gridap)
# v2_vec = A2_mat' \ o_vec
# v2h = FEFunction(gridap.FE_U, v2_vec)

# fig, ax, plt = plot(gridap.Ω, real(u1h), colormap=:bwr)
# Colorbar(fig[1,2], plt)
# ax.aspect = AxisAspect(L/(phys1.LHn[2]+phys1.LHp[2]))
# limits!(ax, -L/2, L/2, -phys1.LHn[2], phys1.LHp[2])
# fig

PhysicalParameters(0.011444781980290685, 1.3304134695650072, 0.5 + 0.0im, 0.5 + 0.0im, 1.0, 1.0e-10, 100.0, [Inf, 700.0], [Inf, 100.0], 200.0)

In [6]:
NO_FIELDS = ZeroTangent()
function g0_pf(pf_vec; kb, phys1, phys2, control, gridap)
    pfh = FEFunction(gridap.FE_Pf, pf_vec)
    pth = (pf -> Threshold(pf; control)) ∘ pfh
    A1_mat = MatrixA(pth, kb; phys=phys1, control, gridap)
    b1_vec = assemble_vector(v->(∫(v)gridap.dΓ_s), gridap.FE_V)
    u1_vec = A1_mat\b1_vec
    u1h = FEFunction(gridap.FE_U, u1_vec)
    
    B_mat = MatrixB(pth, u1h; control, gridap)
    # B_mat = MatrixB(pth, u1fix; control, gridap)
    A2_mat = MatrixA(pth, kb; phys=phys2, control, gridap)
    o_vec = VectorO(1, 1; gridap)
    v2_vec = A2_mat'\o_vec
    g_temp = v2_vec' * B_mat * v2_vec
    # g_temp = v2fix_vec' * B_mat * v2fix_vec
    @assert abs(imag(g_temp) / real(g_temp)) < 1e-6
    real(g_temp)
end

#pf = pf_p0(p0)
function pf_p0(p0; control, gridap)
    p_vec = p_extend(p0; gridap)
    pf_vec = Filter(p_vec; control, gridap)
    pf_vec[pf_vec .< 0] .= 0
    pf_vec[pf_vec .> 1] .= 1.0
    pf_vec
end

# Chain Rule : 
# dg/dpf=dg/dg * dg/dpf
function rrule(::typeof(g0_pf), pf_vec; kb, phys1, phys2, control, gridap)
    function U_pullback(dgdg)
      NO_FIELDS, dgdg * Dg0dpf(pf_vec; kb, phys1, phys2, control, gridap)
    end
    g0_pf(pf_vec; kb, phys1, phys2, control, gridap), U_pullback
end

Dptdpf(pf; control) = control.flag_t ? control.β * (1.0 - tanh(control.β * (pf - control.η))^2) / (tanh(control.β * control.η) + tanh(control.β * (1.0 - control.η))) : 1.0

Dξdpf(pf, nf, nm, α)= 2 * (nf - nm) / (nf + (nm - nf) * Threshold(pf; control))^3 / (1 + control.α * 1im) * Dptdpf(pf; control)

DAdpf(u, v, pfh, kb; phys, control) = ((p -> Dξdpf(p, phys.nf, phys.nm, control.α)) ∘ pfh) * ((∇(v) - 1im * kb * v) ⊙ (∇(u) + 1im * kb * u))

function Dg0dpf(pf_vec; kb, phys1, phys2, control, gridap)
    pfh = FEFunction(gridap.FE_Pf, pf_vec)
    pth = (pf -> Threshold(pf; control)) ∘ pfh
    A1_mat = MatrixA(pth, kb; phys=phys1, control, gridap)
    b1_vec = assemble_vector(v->(∫(v)gridap.dΓ_s), gridap.FE_V)
    u1_vec = A1_mat \ b1_vec
    u1h = FEFunction(gridap.FE_U, u1_vec)
    
    B_mat = MatrixB(pth, u1h; control, gridap)
    # B_mat = MatrixB(pth, u1fix; control, gridap)
    A2_mat = MatrixA(pth, kb; phys=phys2, control, gridap)
    o_vec = VectorO(1, 1; gridap)
    v2_vec = A2_mat' \ o_vec
    v2h = FEFunction(gridap.FE_U, v2_vec)
    # v2h = FEFunction(gridap.FE_U, v2fix_vec)
    
    v2conjh = FEFunction(gridap.FE_U, conj(v2_vec))
    w2_vec =  A2_mat \ (B_mat * v2_vec)
    w2h = FEFunction(gridap.FE_U, w2_vec) 
    
    B_temp = MatrixB(pth, v2h; control, gridap)
    w1_vec = A1_mat' \ (B_temp * u1_vec)
    w1conjh = FEFunction(gridap.FE_U, conj(w1_vec))
    l_temp(dp) = ∫(real(-((pf->Dptdpf(pf; control))∘pfh) * abs2(∇(v2h) ⋅ ∇(u1h))
                         - 2 * 1 * DAdpf(u1h, w1conjh, pfh, kb; phys=phys1, control)
                         - 2 * 1 * DAdpf(w2h, v2conjh, pfh, kb; phys=phys2, control)) * dp)gridap.dΩ_d
    dg0dpf = assemble_vector(l_temp, gridap.FE_Pf)
    return dg0dpf
end

# dg/dp=dg/dpf*dpf/dp
function rrule(::typeof(pf_p0), p0; control, gridap)
  function pf_pullback(dgdpf)
    NO_FIELDS, Dgdp(dgdpf; control, gridap)
  end
  pf_p0(p0; control, gridap), pf_pullback
end

function Dgdp(dgdpf; control, gridap)
    if control.flag_f
        Af = assemble_matrix(gridap.FE_Pf, gridap.FE_Qf) do u, v
            ∫(a_f(control.r, u, v))gridap.dΩ_d + ∫(v * u)gridap.dΩ
        end
        λvec = Af' \ dgdpf
        λh = FEFunction(gridap.FE_Pf, λvec)
        l_temp(dp) = ∫(λh * dp)gridap.dΩ
        return p_extract(assemble_vector(l_temp, gridap.FE_P); gridap)
    else
        return p_extract(dgdpf; gridap)
    end
end


# Final objective function
function g0_p(p0::Vector; kb, phys1, phys2, control, gridap)
    pf_vec = pf_p0(p0; control, gridap)
    g0_pf(pf_vec; kb, phys1, phys2, control, gridap)
end

function g0_p(p0::Vector, grad::Vector; kb, phys1, phys2, control, gridap)
    if length(grad) > 0
        dgdp, = Zygote.gradient(p -> g0_p(p; kb, phys1, phys2, control, gridap), p0)
        grad[:] = dgdp
    end
    g_value = g0_p(p0::Vector; kb, phys1, phys2, control, gridap)
    return g_value
end

g0_p (generic function with 2 methods)

In [7]:
p0 = rand(gridap.np)
δp = rand(gridap.np)*1e-8
grad = zeros(gridap.np)

kb = 0.6 * ω1
g0 = g0_p(p0, grad; kb, phys1, phys2, control, gridap)
g1 = g0_p(p0+δp, []; kb, phys1, phys2, control, gridap)
g1-g0, grad'*δp

(-0.011730948943295516, -0.011637613325097466)

In [52]:
v2fix = v2h
v2fix_vec = v2_vec
u1fix = u1h

SingleFieldFEFunction():
 num_cells: 12250
 DomainStyle: ReferenceDomain()
 Triangulation: UnstructuredGrid()
 Triangulation id: 3118047460089147785

In [152]:
nm2

0.36320816326530625 + 2.677387755102041im